In [1]:
from common.common_imports import *
from sqlalchemy import create_engine, Table, Index, MetaData, select, distinct, exists
from sqlalchemy.sql import and_, desc
from data.time_encoder import TIME_ENCODER
from data.h5py import H5PY_CONTROLLER
from data.history import HISTORY_DATA
from data.info import INFO_DATA


In [2]:
info_data = INFO_DATA()
print(
    info_data.info_tuple_from_index(18)
)
info = info_data.info_array_from_index(18)
print(info)
# info_padded = np.pad(info, (0, history.shape[1]-info.shape[0]), mode='constant', constant_values=np.NINF)

('None', 'Consumer Staples', 'Beverages (Production/Distribution)')
[58.  2. 14.]


In [3]:
class DB:
    def __init__(self) -> None:
        self.engine = create_engine('sqlite:////home/kanesweet/kanesweet/stonks.0.1/stonks.db')
        self.connection = self.engine.connect()
        self.metadata = MetaData(bind=self.engine)
        self.history_table = Table('history', self.metadata, autoload=True, autoload_with=self.engine)
        self.attached = False

db = DB()

In [4]:
h5py = H5PY_CONTROLLER()
tickers_idx = pd.read_csv("data/csv/tickers_idx.csv") #7243
to_remove = []
for i, row in tickers_idx.iterrows():
    symbol = row["ticker"]
    
    query = (select(
                db.history_table.c.Date,
                db.history_table.c.Open,
                db.history_table.c.High,
                db.history_table.c.Low,
                db.history_table.c.Close,
                db.history_table.c.Volume,
            )
            .where(db.history_table.c.Ticker == symbol)
            .order_by(db.history_table.c.Date)
            )

    df = pd.read_sql(query, db.connection)

    df = TIME_ENCODER(df).separate_dates()
    array = df.to_numpy()
    h5py.set_dataset(symbol+"_history", array)

In [ ]:
symbol = "AAPL"
array = h5py.get_dataset(symbol+"_history")
df = pd.DataFrame(array, columns=[
    'Open', 
    'High', 
    'Low', 
    'Close',
    'Volume', 
    'year',
    'month',
    'day'])
print(df)

ValueError: Shape of passed values is (5902, 9), indices imply (5902, 8)